# Types de Données de Base

- Boolean : stocke les valeurs TRUE et FALSE.


- CHAR, VARCHAR, et TEXT : gère différents types de caractères.
    - CHAR(n) : chaîne de longueur fixe.
    - VARCHAR(n) : chaîne de longueur variable.
    - TEXT : texte de longueur illimitée.

- NUMERIC : utilisé pour les nombres avec précision requise, par exemple pour la monnaie.

- Integer : types entiers : SMALLINT, INT, BIGINT.

- DATE : stocke les valeurs de date

- Timestamp : pour les datetimes avec time zone, avec ou sans fuseau horaire.

- Interval : représente une période, comme '2 jours' ou '1 an'.

- TIME : heure précise dans une journée

- UUID : type unique universel, généralement pour des identifiants uniques.

- Array : stocke des tableaux de valeurs, comme des listes de nombres ou de textes.

- hstore : paire clé/valeur pour stocker des données de manière flexible.

- JSON : format json

# Illustration

In [10]:
import os
from dotenv import load_dotenv
# Ajouter aux variables environnementales celles contenues dans le .env
load_dotenv("../../../.env")

USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")


In [11]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Connexion initiale pour créer/supprimer la base de données
conn = psycopg2.connect(
    #dbname="postgres", 
    user=USER, 
    password=PASSWORD, 
    host="localhost", 
    port="5432"
)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

db_name = "example_types"

# Création d'une nouvelle base de données
cur = conn.cursor()
cur.execute(f"DROP DATABASE IF EXISTS {db_name}")
cur.execute(f"CREATE DATABASE {db_name}")
cur.close()
conn.close()

conn = psycopg2.connect(
    dbname=db_name, 
    user=USER, 
    password=PASSWORD, 
    host="localhost", 
    port="5432"
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS table_boolean (id SERIAL PRIMARY KEY, flag BOOLEAN);
CREATE TABLE IF NOT EXISTS table_numeric (id SERIAL PRIMARY KEY, amount NUMERIC);
CREATE TABLE IF NOT EXISTS table_integer (id SERIAL PRIMARY KEY, value INT);
CREATE TABLE IF NOT EXISTS table_date (id SERIAL PRIMARY KEY, date_value DATE);
CREATE TABLE IF NOT EXISTS table_timestamp (id SERIAL PRIMARY KEY, timestamp_value TIMESTAMP WITH TIME ZONE);
CREATE TABLE IF NOT EXISTS table_interval (id SERIAL PRIMARY KEY, interval_value INTERVAL);
CREATE TABLE IF NOT EXISTS table_time (id SERIAL PRIMARY KEY, time_value TIME);
CREATE TABLE IF NOT EXISTS table_uuid (id SERIAL PRIMARY KEY, uuid_value UUID);
CREATE TABLE IF NOT EXISTS table_array (id SERIAL PRIMARY KEY, array_value INT[]);

CREATE TABLE IF NOT EXISTS table_json (id SERIAL PRIMARY KEY, json_value JSON);
""")


# HSTORE : activate extension
# CREATE TABLE IF NOT EXISTS table_hstore (id SERIAL PRIMARY KEY, hstore_value HSTORE);
# https://stackoverflow.com/questions/21909417/pgundefinedobject-error-type-hstore-does-not-exist-but-it-does

import uuid
import json

cur.execute("INSERT INTO table_boolean (flag) VALUES (TRUE)")
cur.execute("INSERT INTO table_numeric (amount) VALUES (100.50)")
cur.execute("INSERT INTO table_integer (value) VALUES (42)")
cur.execute("INSERT INTO table_date (date_value) VALUES ('2023-01-01')")
cur.execute("INSERT INTO table_timestamp (timestamp_value) VALUES ('2023-01-01T12:00:00Z')")
cur.execute("INSERT INTO table_interval (interval_value) VALUES ('1 year 2 months')")
cur.execute("INSERT INTO table_time (time_value) VALUES ('12:00:00')")
cur.execute("INSERT INTO table_uuid (uuid_value) VALUES (%s)", (str(uuid.uuid4()),))
cur.execute("INSERT INTO table_array (array_value) VALUES (ARRAY[1,2,3])")
# cur.execute("INSERT INTO table_hstore (hstore_value) VALUES ('key1=>value1, key2=>value2')")
cur.execute("INSERT INTO table_json (json_value) VALUES (%s)", (json.dumps({"key": "value"}),))

# Validation des modifications
conn.commit()

# Fermeture de la connexion
cur.close()
conn.close()

ObjectInUse: database "example_types" is being accessed by other users
DETAIL:  There is 1 other session using the database.
